In [48]:
import pandas as pd
import numpy as np

print("Imported all libraries")

Imported all libraries


# When downloading the datasets there was a issue where the download would fail after each gigabyte because i think they have some type of limit so i am going to load 2 of the same datasets and make sure that they both match

In [49]:
df1 = pd.read_csv('../../data/original_datasets/weather_data_original.csv')
# df2 = pd.read_csv('../../data/original_datasets/weather_data_original_backup.csv')

# print(f'No lost data from downloads: {df1.equals(df2)}')

In [50]:
# start analyzing the data to asses what needs to be done to have it prepped
df = df1
print(df.isnull().sum())
print('='*100)
print(f'shape: {df.shape}')
print('='*100)
print(df.dtypes)
print('='*100)
df.head(2)

dt                            0
dt_iso                        0
timezone                      0
city_name                     0
lat                           0
lon                           0
temp                          0
visibility             14861048
dew_point                     0
feels_like                    0
temp_min                      0
temp_max                      0
pressure                      0
sea_level              19241397
grnd_level             19241397
humidity                      0
wind_speed                    0
wind_deg                      0
wind_gust              18322567
rain_1h                17684504
rain_3h                19226218
snow_1h                19150527
snow_3h                19241302
clouds_all                    0
weather_id                    0
weather_main                  0
weather_description           0
weather_icon                  0
dtype: int64
shape: (19241397, 28)
dt                       int64
dt_iso                  object
timezon

,dt,dt_iso,timezone,city_name,lat,lon,temp,visibility,dew_point,feels_like,...,wind_gust,rain_1h,rain_3h,snow_1h,snow_3h,clouds_all,weather_id,weather_main,weather_description,weather_icon
0,283996800,1979-01-01 00:00:00 +0000 UTC,-28800,Alameda County,37.601689,-121.719546,6.63,NaN,-0.92,5.57,...,NaN,NaN,NaN,NaN,NaN,0,800,Clear,sky is clear,01d
1,284000400,1979-01-01 01:00:00 +0000 UTC,-28800,Alameda County,37.601689,-121.719546,7.63,NaN,-3.67,7.63,...,NaN,NaN,NaN,NaN,NaN,0,800,Clear,sky is clear,01n


In [51]:
# based on the analysis of the data above i think that it is worth removing the columns with null values each of them are majorinly null
# so it could just end up skewing the data in incrorrect ways the one of the biggest relevance to not remove would be wind and rain
# but since they are so empty there would not be value in keepning them

# we also are goint to remove the dt col and later will change the dt_iso to a datetime object
col_to_drop = ['visibility', 'sea_level', 'grnd_level', 'wind_gust', 'rain_1h', 'rain_3h', 'snow_1h', 'snow_3h', 'dt']
df = df.drop(col_to_drop, axis=1)
print(df.columns)
df.head()


Index(['dt_iso', 'timezone', 'city_name', 'lat', 'lon', 'temp', 'dew_point',
       'feels_like', 'temp_min', 'temp_max', 'pressure', 'humidity',
       'wind_speed', 'wind_deg', 'clouds_all', 'weather_id', 'weather_main',
       'weather_description', 'weather_icon'],
      dtype='object')


,dt_iso,timezone,city_name,lat,lon,temp,dew_point,feels_like,temp_min,temp_max,pressure,humidity,wind_speed,wind_deg,clouds_all,weather_id,weather_main,weather_description,weather_icon
0,1979-01-01 00:00:00 +0000 UTC,-28800,Alameda County,37.601689,-121.719546,6.63,-0.92,5.57,5.69,7.32,1024,58,1.70,14,0,800,Clear,sky is clear,01d
1,1979-01-01 01:00:00 +0000 UTC,-28800,Alameda County,37.601689,-121.719546,7.63,-3.67,7.63,5.80,9.45,1023,43,1.19,59,0,800,Clear,sky is clear,01n
2,1979-01-01 02:00:00 +0000 UTC,-28800,Alameda County,37.601689,-121.719546,6.93,-2.69,6.93,3.75,9.11,1024,49,0.43,111,0,800,Clear,sky is clear,01n
3,1979-01-01 03:00:00 +0000 UTC,-28800,Alameda County,37.601689,-121.719546,-0.11,-4.71,-0.11,-1.63,1.41,1025,68,0.58,156,0,800,Clear,sky is clear,01n
4,1979-01-01 04:00:00 +0000 UTC,-28800,Alameda County,37.601689,-121.719546,1.05,-4.79,1.05,-0.88,2.99,1025,62,1.01,132,0,800,Clear,sky is clear,01n


In [52]:
#change the dt_iso to a datetime object then break the datetime down into year month day hour columns for easier future parsing

df['dt_iso'] = df['dt_iso'].str.replace(' UTC', '')
df['dt_iso'] = pd.to_datetime(df['dt_iso'], format='%Y-%m-%d %H:%M:%S %z', utc=True)
df['Year'] = df['dt_iso'].dt.year
df['Month'] = df['dt_iso'].dt.month
df['Day'] = df['dt_iso'].dt.day
df['Hour'] = df['dt_iso'].dt.hour
df = df.drop('dt_iso', axis=1)

In [53]:
# currently these are all object and they are worth keeping so i will caterogize them numerically so that they can be encoded in training
# weather_main	weather_description	weather_icon
# doing so with one hot encoding


encoded = pd.get_dummies(df[['weather_main',	'weather_description', 'weather_icon']])
df = pd.concat([df, encoded], axis=1)
print(df.shape)
print(df.dtypes)
df.head()


(19241397, 95)
timezone              int64
city_name            object
lat                 float64
lon                 float64
temp                float64
                     ...   
weather_icon_11n       bool
weather_icon_13d       bool
weather_icon_13n       bool
weather_icon_50d       bool
weather_icon_50n       bool
Length: 95, dtype: object


,timezone,city_name,lat,lon,temp,dew_point,feels_like,temp_min,temp_max,pressure,...,weather_icon_09d,weather_icon_09n,weather_icon_10d,weather_icon_10n,weather_icon_11d,weather_icon_11n,weather_icon_13d,weather_icon_13n,weather_icon_50d,weather_icon_50n
0,-28800,Alameda County,37.601689,-121.719546,6.63,-0.92,5.57,5.69,7.32,1024,...,False,False,False,False,False,False,False,False,False,False
1,-28800,Alameda County,37.601689,-121.719546,7.63,-3.67,7.63,5.80,9.45,1023,...,False,False,False,False,False,False,False,False,False,False
2,-28800,Alameda County,37.601689,-121.719546,6.93,-2.69,6.93,3.75,9.11,1024,...,False,False,False,False,False,False,False,False,False,False
3,-28800,Alameda County,37.601689,-121.719546,-0.11,-4.71,-0.11,-1.63,1.41,1025,...,False,False,False,False,False,False,False,False,False,False
4,-28800,Alameda County,37.601689,-121.719546,1.05,-4.79,1.05,-0.88,2.99,1025,...,False,False,False,False,False,False,False,False,False,False
